<a href="https://colab.research.google.com/github/Pranav-Bankar/Data_Science_Lab_SE_A_03/blob/main/Simple_Python_Programs/Experiment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Colab!

### Graph Representation

First, let's define a simple graph using an adjacency list. This dictionary will map each node to a list of its neighboring nodes.

In [ ]:
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("Graph:", graph)

Graph: {'A': ['B', 'C'], 'B': ['A', 'D', 'E'], 'C': ['A', 'F'], 'D': ['B'], 'E': ['B', 'F'], 'F': ['C', 'E']}


### Breadth-First Search (BFS)

BFS explores a graph level by level. It starts at a source node, then visits all its immediate neighbors, then their unvisited neighbors, and so on. It typically uses a queue data structure.

In [ ]:
from collections import deque

def bfs(graph, start_node):
    visited = set() # To keep track of visited nodes
    queue = deque([start_node]) # Queue for BFS traversal
    traversal_order = []

    visited.add(start_node)

    while queue:
        current_node = queue.popleft() # Dequeue a node
        traversal_order.append(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor) # Enqueue unvisited neighbor
    return traversal_order

# Demonstrate BFS
print("BFS Traversal starting from node 'A':", bfs(graph, 'A'))

BFS Traversal starting from node 'A': ['A', 'B', 'C', 'D', 'E', 'F']


### Depth-First Search (DFS)

DFS explores a graph branch by branch. It starts at a source node and explores as far as possible along each branch before backtracking. It typically uses a stack data structure (or recursion, which uses the call stack).

In [ ]:
def dfs(graph, start_node):
    visited = set() # To keep track of visited nodes
    stack = [start_node] # Stack for DFS traversal
    traversal_order = []

    while stack:
        current_node = stack.pop() # Pop a node from the stack

        if current_node not in visited:
            visited.add(current_node)
            traversal_order.append(current_node)

            # Push unvisited neighbors onto the stack (order might affect output but not correctness)
            # Reversing ensures neighbors are processed in the order they appear in adjacency list
            for neighbor in reversed(graph[current_node]):
                if neighbor not in visited:
                    stack.append(neighbor)
    return traversal_order

# Demonstrate DFS
print("DFS Traversal starting from node 'A':", dfs(graph, 'A'))

DFS Traversal starting from node 'A': ['A', 'B', 'D', 'E', 'F', 'C']


### A* Search Algorithm

A* is a popular pathfinding algorithm that finds the shortest path between a starting node and a goal node in a graph. It uses a heuristic function to estimate the cost from the current node to the goal, and it explores nodes that are most likely to lead to the shortest path first. A* combines the benefits of Dijkstra's algorithm (guaranteed shortest path) and greedy best-first search (efficiency through heuristics).

#### Heuristic Function

For the A* algorithm, we need a heuristic function `h(n)` that estimates the cost from node `n` to the goal node. For a simple graph without explicit coordinates, we can use a trivial heuristic like a constant value or a lookup table if distances are known. Since our graph is small and without geographical data, let's assume a simple heuristic where we provide a dictionary of estimated costs from each node to a specific goal (e.g., 'F').

*Note: In a real-world scenario (e.g., maps), the heuristic would be more sophisticated, like Euclidean distance or Manhattan distance.*

In [ ]:
def heuristic(node, goal):
    # A simple, non-admissible heuristic for demonstration purposes.
    # In a real scenario, this would estimate the cost from 'node' to 'goal'.
    # For our small graph, we'll use a lookup.
    h_values = {
        'A': {'F': 5},
        'B': {'F': 4},
        'C': {'F': 2},
        'D': {'F': 6},
        'E': {'F': 1},
        'F': {'F': 0}
    }
    return h_values.get(node, {}).get(goal, float('inf'))

print("Heuristic value from A to F:", heuristic('A', 'F'))

Heuristic value from A to F: 5


#### A* Algorithm Implementation

Now, let's implement the A* algorithm. It will use the `heuristic` function defined above.

In [ ]:
import heapq # For priority queue (min-heap)

def a_star(graph, start_node, goal_node):
    open_set = []  # Priority queue to store (f_score, node)
    heapq.heappush(open_set, (0 + heuristic(start_node, goal_node), start_node)) # (f_score, node)

    g_score = {node: float('inf') for node in graph} # Cost from start to current node
    g_score[start_node] = 0

    came_from = {} # To reconstruct the path

    while open_set:
        f_score, current_node = heapq.heappop(open_set)

        if current_node == goal_node:
            # Reconstruct path
            path = []
            while current_node in came_from:
                path.append(current_node)
                current_node = came_from[current_node]
            path.append(start_node)
            return path[::-1]

        for neighbor in graph[current_node]:
            # Assuming edge cost is 1 for simplicity
            tentative_g_score = g_score[current_node] + 1

            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor, goal_node)
                heapq.heappush(open_set, (f_score, neighbor))

    return None # No path found

# Demonstrate A*
print("A* Path from A to F:", a_star(graph, 'A', 'F'))

A* Path from A to F: ['A', 'C', 'F']


In [ ]:
def print_board(board):
    for i in range(0, 9, 3):
        print(board[i], board[i+1], board[i+2])
    print()

def check_winner(board, player):
    win_conditions = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
        (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
        (0, 4, 8), (2, 4, 6)              # Diagonals
    ]
    for a, b, c in win_conditions:
        if board[a] == board[b] == board[c] == player:
            return True
    return False

def is_game_over(board):
    return check_winner(board, 'X') or check_winner(board, 'O') or ('_' not in board)

def evaluate_state(board):
    if check_winner(board, 'X'):
        return 1  # Maximizer wins
    elif check_winner(board, 'O'):
        return -1 # Minimizer wins
    return 0 # Draw

def get_possible_moves(board):
    return [i for i, spot in enumerate(board) if spot == '_']

def apply_move(board, move, player):
    new_board = list(board)
    new_board[move] = player
    return new_board

def minimax(board, depth, is_maximizing_player):
    if is_game_over(board):
        return evaluate_state(board)

    if is_maximizing_player:
        best_score = -float('inf')
        for move in get_possible_moves(board):
            new_board = apply_move(board, move, 'X')
            score = minimax(new_board, depth + 1, False)
            best_score = max(best_score, score)
        return best_score
    else:
        best_score = float('inf')
        for move in get_possible_moves(board):
            new_board = apply_move(board, move, 'O')
            score = minimax(new_board, depth + 1, True)
            best_score = min(best_score, score)
        return best_score

def find_best_move(board, player):
    best_score = -float('inf') if player == 'X' else float('inf')
    best_move = -1

    for move in get_possible_moves(board):
        new_board = apply_move(board, move, player)
        # If current player is X, the next turn in minimax will be O (minimizing)
        # If current player is O, the next turn in minimax will be X (maximizing)
        score = minimax(new_board, 0, player == 'O')

        if player == 'X': # Maximizing player
            if score > best_score:
                best_score = score
                best_move = move
        else: # Minimizing player
            if score < best_score:
                best_score = score
                best_move = move
    return best_move

# Example 1: Find best move for X on an empty board
empty_board = ['_', '_', '_',
               '_', '_', '_',
               '_', '_', '_']

print("Current Board:")
print_board(empty_board)
best_move_x = find_best_move(empty_board, 'X')
print(f"Best move for 'X' is position: {best_move_x}\n")

# Example 2: Find best move for O on a partially filled board
partially_filled_board = ['X', '_', 'O',
                          '_', 'X', '_',
                          '_', '_', '_']

print("Current Board:")
print_board(partially_filled_board)
best_move_o = find_best_move(partially_filled_board, 'O')
print(f"Best move for 'O' is position: {best_move_o}\n")

# Example 3: A board where X can win
x_can_win_board = ['X', 'O', '_',
                     'X', '_', 'O',
                     '_', '_', '_']

print("Current Board:")
print_board(x_can_win_board)
best_move_x_win = find_best_move(x_can_win_board, 'X')
print(f"Best move for 'X' to win is position: {best_move_x_win}\n")

Current Board:
_ _ _
_ _ _
_ _ _

Best move for 'X' is position: 0

Current Board:
X _ O
_ X _
_ _ _

Best move for 'O' is position: 8

Current Board:
X O _
X _ O
_ _ _

Best move for 'X' to win is position: 4

